In [2]:
import pandas as pd
import difflib
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

In [3]:
df = pd.read_csv('../data/creative_services.csv', skiprows=4)

In [4]:
df.head()

,Unnamed: 0,YEAR,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012
0,ECONOMY,CATEGORY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,"Advertising, market research and public opinio...",..,..,..,..,..,..,0.005589347,0.023603115,..,..
2,Afghanistan,"Architectural, engineering and other technical...",..,..,..,..,..,..,0.123334731,0.083703934,..,..
3,Afghanistan,Research and Development,..,..,..,..,..,..,7.15320346,0.174979874,..,..
4,Afghanistan,"Personal, cultural and recreational services",-,-,-,-,-,-,0.925616871,0.616532382,-,-


In [5]:
df.drop(0, inplace=True)

In [6]:
df.rename(columns={'Unnamed: 0': 'Country Name', 'YEAR': 'Category'}, inplace=True)

In [7]:
import difflib
countries = pd.read_csv('../data/country-codes_csv.csv')
def get_country_iso(name):
    p = difflib.get_close_matches(name, countries['official_name_en'].dropna().unique(), n=1, cutoff=0.5)
    if len(p)>0:
        return countries[countries['official_name_en']==p[0]]['ISO3166-1-Alpha-3'].to_list()[0]
    else:
        return ""

In [8]:
df['country_iso'] = [get_country_iso(x) for x in df['Country Name']]
df[df['country_iso']=='']

,Country Name,Category,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,country_iso
961,"Panama, excluding Canal Zone","Advertising, market research and public opinio...",_,_,_,_,_,_,_,_,_,_,
962,"Panama, excluding Canal Zone","Architectural, engineering and other technical...",_,_,_,_,_,_,_,_,_,_,
963,"Panama, excluding Canal Zone",Research and Development,_,_,_,_,_,_,_,_,_,_,
964,"Panama, excluding Canal Zone","Personal, cultural and recreational services",_,_,_,_,_,_,_,_,_,_,
965,"Panama, excluding Canal Zone",Audiovisual and related services,_,_,_,_,_,_,_,_,_,_,
966,"Panama, excluding Canal Zone","Other other personal, cultural and recreatio...",_,_,_,_,_,_,_,_,_,_,


For most datasets, we are considering only Panama. So we will drop its subdivisions:

In [9]:
df[df['Country Name'].str.contains('Panama')].groupby('Country Name')['country_iso'].last()

Country Name
Panama                          PAN
Panama, Canal Zone              PAN
Panama, excluding Canal Zone       
Name: country_iso, dtype: object

In [10]:
df.count()

Country Name    1422
Category        1422
2003            1422
2004            1422
2005            1422
2006            1422
2007            1422
2008            1422
2009            1422
2010            1422
2011            1422
2012            1422
country_iso     1422
dtype: int64

In [11]:
df.drop(df[df['Country Name'].str.contains('Panama,')].index, inplace=True)

In [12]:
df.count()

Country Name    1410
Category        1410
2003            1410
2004            1410
2005            1410
2006            1410
2007            1410
2008            1410
2009            1410
2010            1410
2011            1410
2012            1410
country_iso     1410
dtype: int64

In [13]:
df.columns[2:-1]

Index(['2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2012'],
      dtype='object')

In [14]:
df_new = df[['Country Name', 'country_iso', 'Category', *df.columns[2:-1]]]

In [15]:
df_new_2 = df_new.melt(id_vars=["Country Name", "country_iso", "Category"], var_name="year", value_name="value")

In [16]:
df_new_2['value'].value_counts()

..             8048
_               960
-               851
0                90
0.265565063       3
               ... 
0.169461596       1
1.269732496       1
0.115790997       1
0.529461711       1
5.31448891        1
Name: value, Length: 3805, dtype: int64

In [17]:
import numpy as np
df_new_2['value'].replace('..', np.nan, inplace=True)
df_new_2['value'].replace('-', np.nan, inplace=True)
df_new_2['value'].replace('_', np.nan, inplace=True)
df_new_2['value'].value_counts()

0              90
0.018355191     3
0.265565063     3
1.048022119     2
0.006086148     2
               ..
0.543038905     1
0.169461596     1
1.269732496     1
0.115790997     1
5.31448891      1
Name: value, Length: 3802, dtype: int64

In [18]:
iso_count = df_new_2.groupby(['country_iso', 'year'])['year'].count()
wrong_isos = iso_count[iso_count>6]
wrong_isos.index.get_level_values(0).unique().to_list()

['CAF',
 'COD',
 'ETH',
 'IDN',
 'IRN',
 'LAO',
 'MNE',
 'NLD',
 'SDN',
 'STP',
 'SYR',
 'TZA']

In [19]:
group_countries = df_new_2.groupby('Country Name')[['Country Name', 'country_iso']].max()
group_countries[group_countries['country_iso'].isin(wrong_isos.index.get_level_values(0).unique().to_list())].sort_values('country_iso')

,Country Name,country_iso
Country Name,,
Central African Republic,Central African Republic,CAF
Czech Republic,Czech Republic,CAF
Dem. Rep. of the Congo,Dem. Rep. of the Congo,COD
"Germany, Democratic Republic of","Germany, Democratic Republic of",COD
Ethiopia,Ethiopia,ETH
Ethiopia (…1991),Ethiopia (…1991),ETH
Indonesia,Indonesia,IDN
Indonesia (…2002),Indonesia (…2002),IDN
Iran (Islamic Republic of),Iran (Islamic Republic of),IRN


In [20]:
df_new_2.drop(df_new_2[df_new_2['Country Name']=='Germany, Democratic Republic of'].index, inplace=True)
df_new_2.drop(df_new_2[df_new_2['Country Name']=='Germany, Federal Republic of'].index, inplace=True)
df_new_2.drop(df_new_2[df_new_2['Country Name']=='Sudan (…2011)'].index, inplace=True)
df_new_2.drop(df_new_2[df_new_2['Country Name']=='Indonesia (…2002)'].index, inplace=True)
df_new_2.drop(df_new_2[df_new_2['Country Name']=='Ethiopia (…1991)'].index, inplace=True)
df_new_2.drop(df_new_2[df_new_2['Country Name']=='Socialist Federative Republic of Yugoslavia'].index, inplace=True)
df_new_2.drop(df_new_2[df_new_2['Country Name']=='Serbia and Montenegro'].index, inplace=True)
df_new_2.drop(df_new_2[df_new_2['Country Name']=='Union of Soviet Socialist Republics'].index, inplace=True)
df_new_2.drop(df_new_2[df_new_2['Country Name']=='Korea, Republic of'].index, inplace=True)
df_new_2.drop(df_new_2[df_new_2['Country Name']=='Yemen, Arab Republic'].index, inplace=True)
df_new_2.drop(df_new_2[df_new_2['Country Name']=='Czechoslovakia'].index, inplace=True)
df_new_2.drop(df_new_2[df_new_2['Country Name']=='Yemen, Democratic'].index, inplace=True)

In [21]:
df_new_2.loc[df_new_2['Country Name']=='Czech Republic','country_iso'] = 'CZE'
df_new_2.loc[df_new_2['Country Name']=='United Kingdom','country_iso'] = 'GBR'
df_new_2.loc[df_new_2['Country Name']=='Netherlands Antilles','country_iso'] = 'ANT'
df_new_2.loc[df_new_2['Country Name']=='China, Taiwan Province of','country_iso'] = 'TWN'

In [22]:
df_new_2[df_new_2['country_iso'].isin(wrong_isos.index.get_level_values(0).unique().to_list())]

,Country Name,country_iso,Category,year,value
234,Central African Republic,CAF,"Advertising, market research and public opinio...",2003,NaN
235,Central African Republic,CAF,"Architectural, engineering and other technical...",2003,NaN
236,Central African Republic,CAF,Research and Development,2003,NaN
237,Central African Republic,CAF,"Personal, cultural and recreational services",2003,NaN
238,Central African Republic,CAF,Audiovisual and related services,2003,NaN
...,...,...,...,...,...
14017,United Republic of Tanzania,TZA,"Architectural, engineering and other technical...",2012,NaN
14018,United Republic of Tanzania,TZA,Research and Development,2012,NaN
14019,United Republic of Tanzania,TZA,"Personal, cultural and recreational services",2012,NaN
14020,United Republic of Tanzania,TZA,Audiovisual and related services,2012,NaN


In [23]:
df_new_3 = df_new_2[['country_iso', 'year', 'Category', 'value']]

In [24]:
df_new_3['tmp_idx'] = df_new_3.index

In [25]:
df_new_3.dropna(inplace=True)

In [26]:
df_new_3

,country_iso,year,Category,value,tmp_idx
6,ALB,2003,"Advertising, market research and public opinio...",0.008614593,6
7,ALB,2003,"Architectural, engineering and other technical...",0.052799117,7
9,ALB,2003,"Personal, cultural and recreational services",0.628587387,9
10,ALB,2003,Audiovisual and related services,0.009448263,10
11,ALB,2003,"Other other personal, cultural and recreatio...",0.619278069,11
...,...,...,...,...,...
14031,URY,2012,"Personal, cultural and recreational services",0.047325702,14031
14047,VEN,2012,"Architectural, engineering and other technical...",3.673469388,14047
14049,VEN,2012,"Personal, cultural and recreational services",0.362811791,14049
14050,VEN,2012,Audiovisual and related services,0.317460318,14050


In [42]:
df_new_31 = df_new_3.set_index(['country_iso','year', 'Category'])

In [44]:
df_new_31.drop(columns=['tmp_idx'], inplace=True)

In [51]:
df_new_31

value
country_iso year Category                                                       
ALB         2003 Advertising, market research and public opinion...  0.008614593
                 Architectural, engineering and other technical ...  0.052799117
                 Personal, cultural and recreational services        0.628587387
                   Audiovisual and related services                  0.009448263
                   Other other personal, cultural and recreation...  0.619278069
...                                                                          ...
URY         2012 Personal, cultural and recreational services        0.047325702
VEN         2012 Architectural, engineering and other technical ...  3.673469388
                 Personal, cultural and recreational services        0.362811791
                   Audiovisual and related services                  0.317460318
                   Other other personal, cultural and recreation...  0.045351474

[4176 rows x 1 columns]

In [45]:
df_new_41 = df_new_31.unstack()

In [57]:
df_new_41.columns = df_new_41.columns.get_level_values(1)

In [73]:
df_new_41.reset_index(inplace=True)

In [74]:
df_new_41.groupby(['country_iso', 'year'])['country_iso'].count().sort_values(ascending=False)

country_iso  year
ZWE          2011    1
FJI          2010    1
             2003    1
             2004    1
             2005    1
                    ..
NGA          2005    1
             2006    1
             2007    1
             2008    1
ABW          2007    1
Name: country_iso, Length: 1235, dtype: int64

In [75]:
df_new_41

Category,country_iso,year,Audiovisual and related services,"Other other personal, cultural and recreational services","Advertising, market research and public opinion polling","Architectural, engineering and other technical services","Personal, cultural and recreational services",Research and Development
0,ABW,2007,NaN,NaN,NaN,NaN,0.212813727,NaN
1,ABW,2008,NaN,NaN,NaN,NaN,0.241148505,NaN
2,ABW,2009,NaN,NaN,NaN,NaN,0.135723432,NaN
3,ABW,2010,NaN,NaN,NaN,NaN,0.090389705,NaN
4,ABW,2011,NaN,NaN,NaN,NaN,0.186552477,NaN
...,...,...,...,...,...,...,...,...
1230,ZWE,2007,NaN,NaN,NaN,NaN,0.721653242,NaN
1231,ZWE,2008,NaN,NaN,NaN,NaN,0.897074756,NaN
1232,ZWE,2009,NaN,NaN,NaN,NaN,0.828908667,NaN
1233,ZWE,2010,NaN,NaN,NaN,NaN,0.745272887,NaN


In [76]:
df_new_41.columns

Index(['country_iso', 'year', '  Audiovisual and related services',
       '  Other other personal, cultural and recreational services',
       'Advertising, market research and public opinion polling',
       'Architectural, engineering and other technical services',
       'Personal, cultural and recreational services',
       'Research and Development'],
      dtype='object', name='Category')

In [77]:
df_new_41.rename(columns={
    '  Audiovisual and related services': 'creative_svc_audiovisual',
    '  Other other personal, cultural and recreational services': 'creative_svc_other_personal_cultural_recreational',
    'Advertising, market research and public opinion polling': 'creative_svc_advertising_mktresearch_polling',
    'Architectural, engineering and other technical services': 'creative_svc_architectural_engineering_technical',
    'Personal, cultural and recreational services': 'creative_svc_personal_cultural_recreational',
    'Research and Development': 'creative_svc_research'
}, inplace=True)

In [81]:
df_new_41.columns

Index(['country_iso', 'year', 'creative_svc_audiovisual',
       'creative_svc_other_personal_cultural_recreational',
       'creative_svc_advertising_mktresearch_polling',
       'creative_svc_architectural_engineering_technical',
       'creative_svc_personal_cultural_recreational', 'creative_svc_research'],
      dtype='object', name='Category')

In [82]:
df_new_41.groupby(['country_iso', 'year'])['country_iso'].count().sort_values(ascending=False)

country_iso  year
ZWE          2011    1
FJI          2010    1
             2003    1
             2004    1
             2005    1
                    ..
NGA          2005    1
             2006    1
             2007    1
             2008    1
ABW          2007    1
Name: country_iso, Length: 1235, dtype: int64

In [83]:
df_new_41.to_csv('../data/converted/creative_services.csv', index=False)

In [84]:
df_saved = pd.read_csv('../data/converted/creative_services.csv')
df_saved

,country_iso,year,creative_svc_audiovisual,creative_svc_other_personal_cultural_recreational,creative_svc_advertising_mktresearch_polling,creative_svc_architectural_engineering_technical,creative_svc_personal_cultural_recreational,creative_svc_research
0,ABW,2007,NaN,NaN,NaN,NaN,0.212814,NaN
1,ABW,2008,NaN,NaN,NaN,NaN,0.241149,NaN
2,ABW,2009,NaN,NaN,NaN,NaN,0.135723,NaN
3,ABW,2010,NaN,NaN,NaN,NaN,0.090390,NaN
4,ABW,2011,NaN,NaN,NaN,NaN,0.186552,NaN
...,...,...,...,...,...,...,...,...
1230,ZWE,2007,NaN,NaN,NaN,NaN,0.721653,NaN
1231,ZWE,2008,NaN,NaN,NaN,NaN,0.897075,NaN
1232,ZWE,2009,NaN,NaN,NaN,NaN,0.828909,NaN
1233,ZWE,2010,NaN,NaN,NaN,NaN,0.745273,NaN
